In [1]:
import re
import numpy as np
import pandas as pd
import logging
from pprint import pprint
import spacy

In [2]:
#gensim stuff
import gensim
import gensim.corpora as corpora
from gensim.models import LdaModel, LdaMulticore
import gensim.downloader as api
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess, lemmatize
from nltk.corpus import stopwords

In [3]:
#plot stuff
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#logging and warnings
import logging
import warnings
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('italian')
stop_words.extend(['co', 'poi', 'https','vaccino','puo','cosi','piu'])
#"no" lasciato apposta per no vax

In [27]:
total0=pd.read_csv('metis_score.csv',names=['user','p'])

In [28]:
total0

,user,p
0,NaN,p
1,GNN1960,0.0
2,ilcurator,0.0
3,mpela61,0.0
4,alessdevara,0.0
5,alessandrograpp,0.0
6,IlZebraapois,0.0
7,mottagiuseppe1,0.0
8,francesco201423,0.0
9,Il_Reggino,0.0


In [29]:
u1=total0.drop(0,axis=0).reset_index().drop("index",axis=1)

In [30]:
u1

,user,p
0,GNN1960,0.0
1,ilcurator,0.0
2,mpela61,0.0
3,alessdevara,0.0
4,alessandrograpp,0.0
5,IlZebraapois,0.0
6,mottagiuseppe1,0.0
7,francesco201423,0.0
8,Il_Reggino,0.0
9,moscam1,0.0


In [31]:
len(u1) #perfetto, quadra con METIS

12111

In [32]:
#database con tweets

In [35]:
tw1 = pd.read_json('../data/twitter_original.json',lines=True)
keyword=tw1['query'][50841]
vaccini1 = tw1['query'] == keyword
df1=tw1[vaccini1]
df11=df1['status']

In [36]:
tw2 = pd.read_json('../data/twitter_stored.json',lines=True)
vaccini2 = tw2['query'] == keyword
df2=tw2[vaccini2]
df22=df2['status']

In [37]:
frames = [df11, df22,]
df = pd.concat(frames)

In [38]:
#estraggo user e testi da df

In [39]:
users=[]
for i in df:
    if ('extended_tweet' in i.keys()):   
        extened=i['extended_tweet']
        if ('full_text' in extened.keys()!=''):                      #controllo che l'utente abbia scritto del testo
            if ('user' in i.keys() != ''):                           #entra nell'user
                user = i['user']
                if ('screen_name' in user.keys()):                        #user_name
                    users.append(user['screen_name'])

In [40]:
text=[]
for i in df:
    if ('extended_tweet' in i.keys()):   
        extened=i['extended_tweet']
        if ('full_text' in extened.keys()):
            text.append(extened['full_text'])

In [41]:
u2=pd.DataFrame(users,columns={'user'})

In [42]:
u2['text']=text

In [43]:
len(u2)

70234

In [44]:
from functools import reduce

In [45]:
def concat(a,b):
    return a + ' ' + b

In [46]:
df_group=u2.groupby('user').apply(lambda df: reduce(concat, df.text)).to_frame('text').reset_index()

In [47]:
df_group

,user,text
0,000120o,"La ministra #Grillo annuncia un ""obbligo fless..."
1,0071959m,@myrtamerlino @RobertoBurioni @Ariachetira Pot...
2,008MORI,"@GuidoCrosetto È normale che sia così, I 5stel..."
3,00MrFrost00,Difendiamo i nostri bambini: #SìVaccini per an...
4,00horus00,@Yi_Benevolence ma che si sono messi questi in...
5,02e872b05cdb475,Difendiamo i nostri bambini: #SìVaccini per an...
6,02giugno,@MARIO_DerFriaul @02giugno ah ah ah io lo mand...
7,05Luglio,@SkyTG24 Non sa come venirne fuori da questa s...
8,06017selci,@emilianofaziosi Ma dai! Quello è un allenator...
9,07Emmedi,"@Aquilante3 @alessia57178448 deficiente, ricor..."


In [48]:
len(df_group)

23428

In [49]:
len(u1)

12111

In [50]:
#df_group=non_df.groupby('user').apply(lambda df: reduce(concat, df.text)).to_frame('text').reset_index()

In [51]:
#total.groupby('user').apply(lambda df: reduce(concat, df.text)).to_frame('text')

In [52]:
total=u1.merge(df_group, on='user', how='inner') #uso inner cosi ho la combinazione di quelli che hanno sia testo che classificazione

In [53]:
total 

,user,p,text
0,ilcurator,0.0,@FabioFranchi1 @NonVaccinato Certo complimenti...
1,francesco201423,0.0,+++ Presentata Proposta di Legge della Maggior...
2,Il_Reggino,0.0,"I vaccini, le mancate nazionalizzazioni, il ri..."
3,alessandro6516,0.0,@07Emmedi @deo_marco Un'altra bella eredità de...
4,francescomorat2,0.0,@angycocco @D77eo @sono_farmaci ... l'abuso di...
5,Inoguci,0.0,@gioca63 @LazarKaganovic1 @RobertoBurioni @Cur...
6,InfoPhotozap,0.0,@Mr_Ozymandias @barbarapucci71 @ingscostanzo @...
7,Imma85611814,0.0,@j4nickgers @ManzoniMauro @VittorioSgarbi @sta...
8,francescoRcan,0.0,@ladyonorato Solo in Italia è possibile una co...
9,morena_m75,0.0,Ho cambiato anche io! Oltretutto o a fare le i...


In [54]:
len(total) #si e' ridotto un bel po' ma ci sta

4791

In [55]:
non_df=total.loc[pd.to_numeric(total['p'])== 0]

In [56]:
non_df

,user,p,text
0,ilcurator,0.0,@FabioFranchi1 @NonVaccinato Certo complimenti...
1,francesco201423,0.0,+++ Presentata Proposta di Legge della Maggior...
2,Il_Reggino,0.0,"I vaccini, le mancate nazionalizzazioni, il ri..."
3,alessandro6516,0.0,@07Emmedi @deo_marco Un'altra bella eredità de...
4,francescomorat2,0.0,@angycocco @D77eo @sono_farmaci ... l'abuso di...
5,Inoguci,0.0,@gioca63 @LazarKaganovic1 @RobertoBurioni @Cur...
6,InfoPhotozap,0.0,@Mr_Ozymandias @barbarapucci71 @ingscostanzo @...
7,Imma85611814,0.0,@j4nickgers @ManzoniMauro @VittorioSgarbi @sta...
8,francescoRcan,0.0,@ladyonorato Solo in Italia è possibile una co...
9,morena_m75,0.0,Ho cambiato anche io! Oltretutto o a fare le i...


In [57]:
data = non_df['text'].values.tolist()

In [58]:
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

In [59]:
data = [re.sub('\s+', ' ', sent) for sent in data]

In [60]:
data = [re.sub("\'", "", sent) for sent in data]

In [61]:
data = [re.sub(r'^https?:\/\/.*[\r\n]*', "", sent, flags=re.MULTILINE) for sent in data] #remove html

In [62]:
data[0]

'Certo complimenti ai sostenitori della scienza non democratica ed il popolo bue zitto con le mutande abbassate a farsi bucare a bisogno!! #fanculo #vaccini #novax La cosa incredibile è che nell’immaginario collettivo #Burioni passi le giornate in laboratorio tra provette e microscopi, non sapendo che è solo in semplice politico! Gli sviluppi e la ricerca dei vaccini li fanno in USA !!! #ignoranza #novax Di cosa sarebbe pluridecorato #Burioni ? È solo un politico #yesman con una laurea in medicina... la ricerca e le scoperte sui vaccini la fanno in USA #SVEGLIATI !! #novax La cosa tragica è che tanti pensano che #Burioni passi la giornate in laboratorio a fare ricerca.... non hanno capito che è solo un politico! #sanita #novax #vaccini'

In [63]:
data = [re.sub(r'[^\w]',' ', sent) for sent in data] #remove punctuation

In [64]:
data[0]

'Certo complimenti ai sostenitori della scienza non democratica ed il popolo bue zitto con le mutande abbassate a farsi bucare a bisogno    fanculo  vaccini  novax La cosa incredibile è che nell immaginario collettivo  Burioni passi le giornate in laboratorio tra provette e microscopi  non sapendo che è solo in semplice politico  Gli sviluppi e la ricerca dei vaccini li fanno in USA      ignoranza  novax Di cosa sarebbe pluridecorato  Burioni   È solo un politico  yesman con una laurea in medicina    la ricerca e le scoperte sui vaccini la fanno in USA  SVEGLIATI     novax La cosa tragica è che tanti pensano che  Burioni passi la giornate in laboratorio a fare ricerca     non hanno capito che è solo un politico   sanita  novax  vaccini'

In [65]:
pprint(data[:1])

['Certo complimenti ai sostenitori della scienza non democratica ed il popolo '
 'bue zitto con le mutande abbassate a farsi bucare a bisogno    fanculo  '
 'vaccini  novax La cosa incredibile è che nell immaginario collettivo  '
 'Burioni passi le giornate in laboratorio tra provette e microscopi  non '
 'sapendo che è solo in semplice politico  Gli sviluppi e la ricerca dei '
 'vaccini li fanno in USA      ignoranza  novax Di cosa sarebbe pluridecorato  '
 'Burioni   È solo un politico  yesman con una laurea in medicina    la '
 'ricerca e le scoperte sui vaccini la fanno in USA  SVEGLIATI     novax La '
 'cosa tragica è che tanti pensano che  Burioni passi la giornate in '
 'laboratorio a fare ricerca     non hanno capito che è solo un politico   '
 'sanita  novax  vaccini']


In [66]:
#tokenize words and clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [67]:
print(data_words[:1])

[['certo', 'complimenti', 'ai', 'sostenitori', 'della', 'scienza', 'non', 'democratica', 'ed', 'il', 'popolo', 'bue', 'zitto', 'con', 'le', 'mutande', 'abbassate', 'farsi', 'bucare', 'bisogno', 'fanculo', 'vaccini', 'novax', 'la', 'cosa', 'incredibile', 'che', 'nell', 'immaginario', 'collettivo', 'burioni', 'passi', 'le', 'giornate', 'in', 'laboratorio', 'tra', 'provette', 'microscopi', 'non', 'sapendo', 'che', 'solo', 'in', 'semplice', 'politico', 'gli', 'sviluppi', 'la', 'ricerca', 'dei', 'vaccini', 'li', 'fanno', 'in', 'usa', 'ignoranza', 'novax', 'di', 'cosa', 'sarebbe', 'pluridecorato', 'burioni', 'solo', 'un', 'politico', 'yesman', 'con', 'una', 'laurea', 'in', 'medicina', 'la', 'ricerca', 'le', 'scoperte', 'sui', 'vaccini', 'la', 'fanno', 'in', 'usa', 'svegliati', 'novax', 'la', 'cosa', 'tragica', 'che', 'tanti', 'pensano', 'che', 'burioni', 'passi', 'la', 'giornate', 'in', 'laboratorio', 'fare', 'ricerca', 'non', 'hanno', 'capito', 'che', 'solo', 'un', 'politico', 'sanita', 'no

In [68]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [69]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('it', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [70]:
print(data_lemmatized[:1])

[['certo', 'complimento', 'al', 'sostenitore', 'della', 'scienza', 'non', 'democratico', 'popolare', 'bue', 'zitto', 'mutanda', 'abbassare', 'farsi', 'bucare', 'bisognare', 'fanculo', 'vaccino', 'cosa', 'incredibile', 'nell', 'immaginario', 'collettivo', 'burioni', 'passo', 'giornata', 'laboratorio', 'provetto', 'microscopio', 'non', 'sapere', 'solere', 'semplice', 'politico', 'sviluppo', 'ricercare', 'vaccino', 'fare', 'uso', 'ignoranza', 'novax', 'pluridecorato', 'burioni', 'solere', 'politico', 'yesman', 'laureare', 'medicina', 'ricercare', 'scoprire', 'vaccino', 'fare', 'uso', 'svegliare', 'cosa', 'tragico', 'pensare', 'burioni', 'passo', 'giornata', 'laboratorio', 'fare', 'ricercare', 'non', 'capitare', 'solere', 'politico', 'sanita', 'novax', 'vaccino']]


In [71]:
for i in data_lemmatized: #lower case
    for j in i:
        j.lower()

In [72]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [73]:
data_words_nostops = remove_stopwords(data_lemmatized)

In [74]:
print(data_words_nostops[:1])

[['certo', 'complimento', 'sostenitore', 'scienza', 'democratico', 'popolare', 'bue', 'zitto', 'mutanda', 'abbassare', 'farsi', 'bucare', 'bisognare', 'fanculo', 'cosa', 'incredibile', 'immaginario', 'collettivo', 'burioni', 'passo', 'giornata', 'laboratorio', 'provetto', 'microscopio', 'sapere', 'solere', 'semplice', 'politico', 'sviluppo', 'ricercare', 'fare', 'uso', 'ignoranza', 'novax', 'pluridecorato', 'burioni', 'solere', 'politico', 'yesman', 'laureare', 'medicina', 'ricercare', 'scoprire', 'fare', 'uso', 'svegliare', 'cosa', 'tragico', 'pensare', 'burioni', 'passo', 'giornata', 'laboratorio', 'fare', 'ricercare', 'capitare', 'solere', 'politico', 'sanita', 'novax']]


In [53]:
#bigrams and trigramns non servono!

In [34]:
#non serve
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

#creating bigram and trigram
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words_nostops, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words_nostops], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words_nostops[0]]])

In [75]:
#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_words_nostops)

# Create Corpus
texts = data_words_nostops

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]



In [76]:
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 3), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 3), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 2), (25, 1), (26, 1), (27, 3), (28, 1), (29, 1), (30, 3), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1)]]


In [77]:
len(id2word)

31063

In [78]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('abbassare', 1),
  ('bisognare', 1),
  ('bucare', 1),
  ('bue', 1),
  ('burioni', 3),
  ('capitare', 1),
  ('certo', 1),
  ('collettivo', 1),
  ('complimento', 1),
  ('cosa', 2),
  ('democratico', 1),
  ('fanculo', 1),
  ('fare', 3),
  ('farsi', 1),
  ('giornata', 2),
  ('ignoranza', 1),
  ('immaginario', 1),
  ('incredibile', 1),
  ('laboratorio', 2),
  ('laureare', 1),
  ('medicina', 1),
  ('microscopio', 1),
  ('mutanda', 1),
  ('novax', 2),
  ('passo', 2),
  ('pensare', 1),
  ('pluridecorato', 1),
  ('politico', 3),
  ('popolare', 1),
  ('provetto', 1),
  ('ricercare', 3),
  ('sanita', 1),
  ('sapere', 1),
  ('scienza', 1),
  ('scoprire', 1),
  ('semplice', 1),
  ('solere', 3),
  ('sostenitore', 1),
  ('svegliare', 1),
  ('sviluppo', 1),
  ('tragico', 1),
  ('uso', 2),
  ('yesman', 1),
  ('zitto', 1)]]

In [79]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, #5 topics
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True,
                                           eval_every=None) #evaluated every mettere None

In [94]:
lda_model.save('lda_model_provax')

In [84]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.324889776667398

Coherence Score:  0.40299759520454737


In [80]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/acossard/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.332196  0.142480       1        1  83.305527
4     -0.079653 -0.306350       2        1  14.306895
3      0.163229  0.064306       3        1   1.347806
1      0.129613  0.055072       4        1   0.648776
2      0.119007  0.044492       5        1   0.391005, topic_info=      Category         Freq             Term        Total  loglift  logprob
term                                                                       
82     Default  1869.000000          leggere  1869.000000  30.0000  30.0000
1615   Default  1037.000000  libertadiscelta  1037.000000  29.0000  29.0000
1573   Default   955.000000              ddl   955.000000  28.0000  28.0000
121    Default  1112.000000           scuola  1112.000000  27.0000  27.0000
73     Default  8405.000000           essere  8405.000000  26.0000  26.0000
921    Default  1383.000000        obbligare  1383.000000  25.0000  25.0000
316    Default   694.000000        governare   694.000000  24.0000  24.0000
2291   Default   276.000000           provax   276.000000  23.0000  23.0000
86     Default   970.000000         lorenzin   970.000000  22.0000  22.0000
12     Default  5917.000000             fare  5917.000000  21.0000  21.0000
641    Default  1842.000000          bambino  1842.000000  20.0000  20.0000
232    Default  1872.000000               no  1872.000000  19.0000  19.0000
81     Default   479.000000           legare   479.000000  18.0000  18.0000
3772   Default   396.000000        settembre   396.000000  17.0000  17.0000
1697   Default   342.000000     tuttiascuola   342.000000  16.0000  16.0000
175    Default   405.000000              via   405.000000  15.0000  15.0000
95     Default   600.000000            nuovo   600.000000  14.0000  14.0000
2182   Default   277.000000            corpo   277.000000  13.0000  13.0000
133    Default   313.000000           votare   313.000000  12.0000  12.0000
52     Default  3031.000000            avere  3031.000000  11.0000  11.0000
1602   Default   436.000000         grillare   436.000000  10.0000  10.0000
3023   Default   259.000000     parlamentare   259.000000   9.0000   9.0000
109    Default   247.000000       promettere   247.000000   8.0000   8.0000
4167   Default   115.000000        devastare   115.000000   7.0000   7.0000
3645   Default    93.000000           vaxxed    93.000000   6.0000   6.0000
4556   Default   229.000000           killer   229.000000   5.0000   5.0000
2173   Default    99.000000          tossico    99.000000   4.0000   4.0000
968    Default   371.000000       ministrare   371.000000   3.0000   3.0000
511    Default   218.000000           senato   218.000000   2.0000   2.0000
63     Default   223.000000        decretare   223.000000   1.0000   1.0000
...        ...          ...              ...          ...      ...      ...
520     Topic5     3.075704              are     3.523182   5.4084  -6.0852
551     Topic5     2.276870             have     2.672978   5.3838  -6.3860
21841   Topic5     2.175832         fbuaoeeh     2.591032   5.3696  -6.4314
22378   Topic5     2.037037      translation     2.432853   5.3666  -6.4973
16841   Topic5     1.640625         randieri     2.036232   5.3282  -6.7137
24317   Topic5     1.666668           mmwave     2.083333   5.3211  -6.6979
21544   Topic5     2.969022         ahahahah     3.759778   5.3081  -6.1205
21863   Topic5     1.373482           taiwan     1.787775   5.2806  -6.8914
4076    Topic5     1.319038          because     1.726480   5.2750  -6.9319
24271   Topic5     1.325268       hobbesiano     1.745691   5.2687  -6.9272
24252   Topic5     1.325268           cyborg     1.745691   5.2687  -6.9272
5721    Topic5     1.154429           vaccin     1.547326   5.2513  -7.0652
4080    Topic5     1.190677         children     1.596463   5.2509  -7.0342
16921   Topic5     1.067196             safe     1.455802   5.2337  -7.1437
6093   

In [81]:
top_words_per_topic = []
for t in range(lda_model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 15)])

In [82]:
pprint(top_words_per_topic)

[(0, 'essere', 0.029196342),
 (0, 'fare', 0.020550445),
 (0, 'avere', 0.010527677),
 (0, 'vaccinare', 0.009677702),
 (0, 'solere', 0.008097921),
 (0, 'perche', 0.0076244925),
 (0, 'dire', 0.006617733),
 (0, 'vaccinazione', 0.0063193166),
 (0, 'no', 0.0053666574),
 (0, 'stare', 0.0053069214),
 (0, 'bambino', 0.0051462906),
 (0, 'sapere', 0.005073884),
 (0, 'obbligatorio', 0.005053446),
 (0, 'anno', 0.004914648),
 (0, 'parlare', 0.004404081),
 (1, 'vaxxed', 0.012389016),
 (1, 'lera', 0.012108496),
 (1, 'attore', 0.008309385),
 (1, 'allautismo', 0.008064901),
 (1, 'trucco', 0.007203798),
 (1, 'iftsfgqpcq', 0.0071151326),
 (1, 'cinema', 0.006923311),
 (1, 'lacrima', 0.0068040323),
 (1, 'lightblu', 0.0061769546),
 (1, 'dsxatk', 0.0059308237),
 (1, 'osh', 0.0059308237),
 (1, 'accanto', 0.005595939),
 (1, 'conclamare', 0.005275365),
 (1, 'progredito', 0.004973591),
 (1, 'provax', 0.0048171394),
 (2, 'imercati', 0.004978028),
 (2, 'is', 0.0023346702),
 (2, 'c_randieri', 0.0022777522),
 (2, 'ag

In [83]:
with open('top_words_antivax.txt', 'w') as f:
    for i in top_words_per_topic:
        f.write(str(i[0])+"\t"+str(i[1])+'\n')